The best parameters for the models were as follows:
MCF/DropSeq: 'Neural Network': {'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.1}
HCC/DropSeq: Ensemble: of Logistic Regression, SVM, and Random Forest with their best parameters:
'Logistic Regression': {'C': 0.1, 'penalty': 'l2'}, 'Random Forest': {'max_depth': 100, 'n_estimators': 200}, 'SVM': {'C': 10, 'kernel': 'rbf'}
MCF/SmartSeq: 'Random Forest': {'max_depth': 10, 'n_estimators': 50}
HCC/SmartSeq: 'Random Forest': {'max_depth': 10, 'n_estimators': 50}

models = {'Logistic Regression': LogisticRegression(solver='liblinear', random_state=42),
          'KNN': KNeighborsClassifier(),
          'Random Forest': RandomForestClassifier(random_state=42),
          'SVM': SVC(probability=True, random_state=42),
          'Neural Network': MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
          }


In [29]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from  sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [30]:
#laptop
MCF_DS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
HCC_DS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
MCF_SS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\MCF7_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
HCC_SS = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\HCC1806_SmartS_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)

In [50]:
#testing daata
MCF_DS_test = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_train.txt", delimiter="\ ", engine='python', index_col=0)
HCC_DS_test = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_test_anonim.txt", delimiter="\ ", engine='python', index_col=0)
MCF_SS_test = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\MCF7_SmartS_Filtered_Normalised_3000_Data_test_anonim.txt", delimiter="\ ", engine='python', index_col=0)
HCC_SS_test = pd.read_csv("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\HCC1806_SmartS_Filtered_Normalised_3000_Data_test_anonim.txt", delimiter="\ ", engine='python', index_col=0)

In [32]:
def oxia(df):
    new_row = {col: None for col in df.columns}
    for col in df.columns:
        if '_Nor' in col:
            new_row[col] = 1
        elif '_Hyp' in col:
            new_row[col] = 0
    return pd.DataFrame(new_row, index=['oxia'])


In [33]:
#Oxia them and split them into X and y
y_MCF_DS = oxia(MCF_DS).T
y_HCC_DS = oxia(HCC_DS).T
y_MCF_SS = oxia(MCF_SS)
y_HCC_SS = oxia(HCC_SS)

In [34]:
y_MCF_DS

,oxia
"""AAAAACCTATCG_Normoxia""",1
"""AAAACAACCCTA_Normoxia""",1
"""AAAACACTCTCA_Normoxia""",1
"""AAAACCAGGCAC_Normoxia""",1
"""AAAACCTAGCTC_Normoxia""",1
...,...
"""TTTTCGCGTAGA_Hypoxia""",0
"""TTTTCGTCCGCT_Hypoxia""",0
"""TTTTCTCCGGCT_Hypoxia""",0
"""TTTTGTTCAAAG_Hypoxia""",0


In [35]:
y_HCC_SS

,"""output.STAR.PCRPlate1G12_Normoxia_S32_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G1_Hypoxia_S102_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G2_Hypoxia_S2_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G3_Hypoxia_S7_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G4_Hypoxia_S107_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G7_Normoxia_S118_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G8_Normoxia_S19_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1G9_Normoxia_S121_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1H1_Hypoxia_S103_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate1H2_Hypoxia_S3_Aligned.sortedByCoord.out.bam""",...,"""output.STAR.PCRPlate4G12_Normoxia_S243_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4G1_Hypoxia_S193_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4G2_Hypoxia_S198_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4G6_Hypoxia_S232_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4G7_Normoxia_S204_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4H10_Normoxia_S210_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4H11_Normoxia_S214_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4H2_Hypoxia_S199_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4H7_Normoxia_S205_Aligned.sortedByCoord.out.bam""","""output.STAR.PCRPlate4H9_Normoxia_S236_Aligned.sortedByCoord.out.bam"""
oxia,1,0,0,0,0,1,1,1,0,0,...,1,0,0,0,1,1,1,0,1,1


In [36]:
MCF_DS = MCF_DS.T
HCC_DS = HCC_DS.T
HCC_SS = HCC_SS.T
MCF_SS = MCF_SS.T
y_MCF_SS = y_MCF_SS.T
y_HCC_SS = y_HCC_SS.T

In [53]:
MCF_DS_test = MCF_DS_test.T
MCF_DS_test

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""MROH1""","""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3"""
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
MCF_DS

,"""MALAT1""","""MT-RNR2""","""NEAT1""","""H1-5""","""TFF1""","""MT-RNR1""","""H4C3""","""GDF15""","""KRT81""","""MT-CO3""",...,"""MROH1""","""SKIDA1""","""MICALL1""","""RARG""","""MYO1F""","""BRWD1-AS2""","""RPS19BP1""","""AUNIP""","""TNK2""","""SUDS3"""
"""AAAAACCTATCG_Normoxia""",1,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACAACCCTA_Normoxia""",3,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACACTCTCA_Normoxia""",3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCAGGCAC_Normoxia""",6,2,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""AAAACCTAGCTC_Normoxia""",4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""TTTTCGCGTAGA_Hypoxia""",0,0,0,0,3,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCGTCCGCT_Hypoxia""",1,0,0,0,4,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
"""TTTTCTCCGGCT_Hypoxia""",0,0,0,1,2,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
"""TTTTGTTCAAAG_Hypoxia""",0,0,0,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
HCC_DS_test = HCC_DS_test.T
HCC_DS_test

,"""H1-5""","""MALAT1""","""MT-RNR2""","""ARVCF""","""BCYRN1""","""ATXN7L2""","""IGFBP3""","""H1-3""","""CTIF""","""RNF123""",...,"""BATF3""","""CDKN3""","""DLD""","""PMPCA""","""ZNF165""","""SCCPDH""","""NTAN1""","""CLIP2""","""DUSP23""","""ZNF682"""
"""1""",0,3,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""2""",0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""3""",2,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
"""4""",0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
"""5""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""3667""",1,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""3668""",0,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
"""3669""",0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""3670""",0,0,0,0,3,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
MCF_SS_test = MCF_SS_test.T
MCF_SS_test

,"""CYP1B1""","""CYP1B1-AS1""","""CYP1A1""","""NDRG1""","""DDIT4""","""PFKFB3""","""HK2""","""AREG""","""MYBL2""","""ADM""",...,"""CD27-AS1""","""DNAI7""","""MAFG""","""LZTR1""","""BCO2""","""GRIK5""","""SLC25A27""","""DENND5A""","""CDK5R1""","""FAM13A-AS1"""
"""1""",492,253,0,1157,6805,5447,2662,2448,0,58,...,0,0,193,0,0,0,0,285,0,1
"""2""",7199,3245,7181,1857,20731,4374,5490,0,0,2150,...,0,0,187,0,0,0,0,55,0,0
"""3""",12,11,1,5,147,301,9,647,1890,1,...,0,0,28,0,0,0,0,1,0,0
"""4""",373,187,0,0,43,37,0,0,580,0,...,21,0,131,0,0,0,0,3,0,0
"""5""",31,13,0,0,0,44,8,0,554,0,...,43,0,30,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"""59""",287,126,666,3270,25776,12144,1119,0,0,678,...,0,0,473,0,0,0,0,315,0,26
"""60""",15220,6316,1991,750,12176,7488,3172,2030,0,64,...,0,0,36,0,0,0,34,78,0,0
"""61""",21998,8898,21329,1498,5144,1146,914,0,0,0,...,69,0,0,0,0,0,0,0,0,35
"""62""",39,17,1,29,20,0,0,88,1399,0,...,12,0,0,0,0,0,0,0,0,0


In [59]:
HCC_SS_test = HCC_SS_test.T
HCC_SS_test

,"""DDIT4""","""ANGPTL4""","""CALML5""","""KRT14""","""CCNB1""","""IGFBP3""","""AKR1C2""","""KRT6A""","""NDRG1""","""KRT4""",...,"""MST1R""","""ZYG11A""","""NRG1""","""RBMS3""","""VCPIP1""","""LINC02693""","""OR8B9P""","""NEAT1""","""ZDHHC23""","""ODAD2"""
"""1""",0,0,0,169,233,227,521,577,0,0,...,0,0,176,0,0,48,0,118,6,0
"""2""",2475,0,0,0,3537,843,214,0,3,0,...,106,0,333,0,18,65,0,39,0,0
"""3""",9088,2143,0,0,124,0,0,0,415,0,...,299,0,0,45,0,0,0,172,0,0
"""4""",6909,3086,0,909,78,466,508,193,453,0,...,319,0,95,167,0,5,0,31,0,10
"""5""",13655,2196,0,1,1,14688,3872,184,884,176,...,0,0,1,0,0,0,0,224,0,0
"""6""",7684,1619,6,74,0,1205,79,92,696,208,...,0,0,0,0,0,50,0,174,0,0
"""7""",13038,1917,0,108,991,134,681,171,524,0,...,255,0,144,0,0,0,0,11,46,0
"""8""",640,1,0,1,128,168,2601,2421,1,0,...,1,0,187,0,51,14,0,86,0,0
"""9""",3,1,0,1794,256,1075,3335,4781,1,4729,...,71,0,1,0,0,0,0,61,0,0
"""10""",542,81,0,99,180,183,660,93,77,0,...,47,49,4,26,0,151,0,15,31,0


In [66]:
#{'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.1},
#           'Neural Network': MLPClassifier(random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')

#MCF/DropSeq
mlp = MLPClassifier(hidden_layer_sizes=(50, 50), learning_rate_init=0.1, random_state=42, batch_size='auto', max_iter=1000000, solver='sgd')
mlp.fit(MCF_DS, y_MCF_DS)
y_pred = mlp.predict(MCF_DS_test)

C:\Users\camer\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
y_pred

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [67]:
#check none are missing
np.isnan(y_pred).sum()

0

In [68]:
#save the predictions "C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\
np.savetxt("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\MCF7_Filtered_Normalised_3000_Data_test_predictions.txt", y_pred, fmt='%s')

In [69]:
#Ensemble: of Logistic Regression, SVM, and Random Forest with their best parameters:
#'Logistic Regression': {'C': 0.1, 'penalty': 'l2'}, 'Random Forest': {'max_depth': 100, 'n_estimators': 200}, 'SVM': {'C': 10, 'kernel': 'rbf'}
#HCC/DropSeq
lr = LogisticRegression(C=0.1, penalty='l2', solver='liblinear', random_state=42)
rf = RandomForestClassifier(max_depth=100, n_estimators=200, random_state=42)
svm = SVC(C=10, kernel='rbf', probability=True, random_state=42)
ensemble = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svm', svm)], voting='soft')
ensemble.fit(HCC_DS, y_HCC_DS)
y_pred = ensemble.predict(HCC_DS_test)

C:\Users\camer\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\camer\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [70]:
y_pred

array([1, 1, 1, ..., 0, 0, 1], dtype=int64)

In [71]:
#check none are missing
np.isnan(y_pred).sum()

0

In [72]:
#save the predictions "C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\
np.savetxt("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\DropSeq\\HCC1806_Filtered_Normalised_3000_Data_test_predictions.txt", y_pred, fmt='%s')

In [73]:
#MCF/SmartSeq
rf = RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)
rf.fit(MCF_SS, y_MCF_SS)
y_pred = rf.predict(MCF_SS_test)

C:\Users\camer\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [74]:
y_pred

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
      dtype=int64)

In [75]:
#check none are missing
np.isnan(y_pred).sum()

0

In [76]:
#save the predictions "C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\
np.savetxt("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\MCF7_SmartS_Filtered_Normalised_3000_Data_test_predictions.txt", y_pred, fmt='%s')

In [77]:
#HCC/SmartSeq
rf = RandomForestClassifier(max_depth=10, n_estimators=50, random_state=42)
rf.fit(HCC_SS, y_HCC_SS)
y_pred = rf.predict(HCC_SS_test)

C:\Users\camer\PycharmProjects\AI-Lab\venv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [78]:
y_pred

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1], dtype=int64)

In [79]:
#check none are missing
np.isnan(y_pred).sum()

0

In [80]:
#save the predictions "C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\
np.savetxt("C:\\Users\\camer\\Desktop\\class stuff\\Semester 4\\ai lab\\SmartSeq\\HCC1806_SmartS_Filtered_Normalised_3000_Data_test_predictions.txt", y_pred, fmt='%s')